In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3) #28 26 3   (Input size, Output size and receptive field)
        self.Batch1 = nn.BatchNorm2d(16)
        self.Drop1 = nn.Dropout(0.1)

        self.conv2 = nn.Conv2d(16, 16, 3) #26 24 5 (Going for maxpooling at 5 because for MNIST we can start to see the edges appear within 5 pixels - not recommended for other detasets)
        self.Batch2 = nn.BatchNorm2d(16)
        self.Drop2 = nn.Dropout(0.1)

        self.conv3 = nn.Conv2d(16,16,1) #24 24 5 (No receptive field change)
        self.pool1 = nn.MaxPool2d(2, 2) #24 12 10 (Max pool doubles the receptive field - not True , but used as such for beginner sessions)
        self.conv4 = nn.Conv2d(16, 16, 3) #12 10 12
        self.Batch4 = nn.BatchNorm2d(16)
        self.Drop4 = nn.Dropout(0.1)

        self.conv5 = nn.Conv2d(16, 16, 3) #10 8 14
        self.Batch5 = nn.BatchNorm2d(16)
        self.Drop5 = nn.Dropout(0.1)

        self.conv6 = nn.Conv2d(16, 16, 3) #8 6 16
        self.Batch6 = nn.BatchNorm2d(16)
        self.Drop6 = nn.Dropout(0.1)

        self.conv7 = nn.Conv2d(16, 32, 3) #6 4 18
        self.Drop7 = nn.Dropout(0.1)

        self.conv8 = nn.Conv2d(32 ,10, 4) #(No relu used with the last latyer ,Batch Norm also not being used before the last layer  )
 
    def forward(self, x):
        x= F.relu(self.conv1(x))
        x=self.Batch1(x)
        x=self.Drop1(x)

        x= F.relu(self.conv2(x))
        x=self.Batch2(x)
        x=self.Drop2(x)

        x=self.pool1(F.relu(self.conv3(x)))

        x= F.relu(self.conv4(x))
        x=self.Batch4(x)
        x=self.Drop4(x)

        x= F.relu(self.conv5(x))
        x=self.Batch5(x)
        x=self.Drop5(x)

        x= F.relu(self.conv6(x))
        x=self.Batch6(x)
        x=self.Drop6(x)

        x= F.relu(self.conv7(x))
       #x=self.Batch7(x)
        x=self.Drop7(x)

        x=self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 24, 24]             272
         MaxPool2d-8           [-1, 16, 12, 12]               0
            Conv2d-9           [-1, 16, 10, 10]           2,320
      BatchNorm2d-10           [-1, 16, 10, 10]              32
          Dropout-11           [-1, 16, 10, 10]               0
           Conv2d-12             [-1, 16, 8, 8]           2,320
    

<ipython-input-3-2dd48abca5a2>:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [5]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 379316204.66it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 108447353.47it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 230706588.27it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 21550371.91it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-3-2dd48abca5a2>:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.03291691467165947 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.06it/s]



Test set: Average loss: 0.0469, Accuracy: 9858/10000 (99%)



loss=0.07784946262836456 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.02it/s]



Test set: Average loss: 0.0338, Accuracy: 9901/10000 (99%)



loss=0.04892730712890625 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.90it/s]


...


Test set: Average loss: 0.0213, Accuracy: 9935/10000 (99%)



loss=0.029037604108452797 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.96it/s]



Test set: Average loss: 0.0222, Accuracy: 9933/10000 (99%)



loss=0.014352451078593731 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.84it/s]



Test set: Average loss: 0.0213, Accuracy: 9942/10000 (99%)

